In [2]:
import os
import json

data_path = '/datasets/TACO-master/data'
anno_file = 'annotations.json'

with open(os.path.join(data_path, anno_file), 'r') as f:
    anno = json.load(f)
    
len(anno)

7

In [4]:
list(anno.keys())

['info',
 'images',
 'annotations',
 'scene_annotations',
 'licenses',
 'categories',
 'scene_categories']

In [39]:
anno['images'][2]

{'id': 2,
 'width': 1537,
 'height': 2049,
 'file_name': 'batch_1/000010.jpg',
 'license': None,
 'flickr_url': 'https://farm66.staticflickr.com/65535/40888872753_08ffb24902_o.png',
 'coco_url': None,
 'date_captured': None,
 'flickr_640_url': 'https://farm66.staticflickr.com/65535/40888872753_631ab0f441_z.jpg'}

In [49]:
#anno['annotations'][5]

In [48]:
from pycocotools.coco import COCO


In [52]:
dataDir= '/datasets/TACO-master/data'
dataType='val2017'
annFile='{}/annotations.json'.format(dataDir)


In [53]:
coco=COCO(annFile)


loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [ ]:
coco.